In [5]:
import requests
import pandas as pd
from datetime import datetime, timedelta

# 1. Generar fechas de los últimos 2 meses
fecha_fin = datetime.now().date()
fecha_ini = fecha_fin - timedelta(days=60)
fechas = pd.date_range(start=fecha_ini, end=fecha_fin, freq='D')

# 2. Extraer tipo de cambio para cada fecha
data = []
for fecha in fechas:
    fecha_str = fecha.strftime('%Y-%m-%d')
    url = f'https://api.apis.net.pe/v1/tipo-cambio-sunat?fecha={fecha_str}'
    response = requests.get(url)
    if response.status_code == 200:
        tipo_cambio = response.json()
        tipo_cambio['fecha'] = fecha_str
        data.append(tipo_cambio)

# 3. Crear DataFrame
df_cambio = pd.DataFrame(data)
df_cambio['fecha'] = pd.to_datetime(df_cambio['fecha'], errors='coerce')

df_cambio

,compra,venta,origen,moneda,fecha
0,3.706,3.716,SUNAT,USD,2025-01-27
1,3.715,3.728,SUNAT,USD,2025-02-01
2,3.712,3.720,SUNAT,USD,2025-02-06
3,3.707,3.719,SUNAT,USD,2025-02-11
4,3.702,3.712,SUNAT,USD,2025-02-16
5,3.681,3.687,SUNAT,USD,2025-02-21
6,3.678,3.686,SUNAT,USD,2025-02-26
7,3.671,3.683,SUNAT,USD,2025-03-03
8,3.654,3.659,SUNAT,USD,2025-03-08
9,3.658,3.667,SUNAT,USD,2025-03-13


In [6]:
# --- Completitud ---
completitud = {
    'Total registros': len(df_cambio),
    'Fechas únicas': df_cambio['fecha'].nunique(),
    'Nulos en campos': df_cambio.isnull().sum().to_dict(),
    'Porcentaje de días sin datos': round((len(fechas) - len(df_cambio)) / len(fechas) * 100, 2)
}

# --- Unicidad ---
unicidad = {
    'Duplicados por fecha': df_cambio.duplicated('fecha').sum(),
    '¿Fechas únicas = registros?': df_cambio['fecha'].nunique() == len(df_cambio)
}

# --- Consistencia ---
consistencia = {
    'Tipo de cambio < 2.0 (inválido)': ((df_cambio['venta'] < 2) | (df_cambio['compra'] < 2)).sum(),
    'Tipo de cambio > 5.0 (inválido)': ((df_cambio['venta'] > 5) | (df_cambio['compra'] > 5)).sum(),
    'Compra mayor que venta': (df_cambio['compra'] > df_cambio['venta']).sum()
}

# --- Validez ---
validez = {
    'Mínimo valor de venta': df_cambio['venta'].min(),
    'Máximo valor de venta': df_cambio['venta'].max(),
    'Mínimo valor de compra': df_cambio['compra'].min(),
    'Máximo valor de compra': df_cambio['compra'].max()
}

# --- Actualidad ---
actualidad = {
    'Primera fecha': df_cambio['fecha'].min(),
    'Última fecha': df_cambio['fecha'].max(),
    'Cantidad de días cubiertos': (df_cambio['fecha'].max() - df_cambio['fecha'].min()).days + 1
}

# 5. Mostrar indicadores
print("\nIndicadores de Calidad - Tipo de Cambio SUNAT (últimos 2 meses)\n")

print("Completitud:")
for k, v in completitud.items():
    print(f" - {k}: {v}")

print("\nUnicidad:")
for k, v in unicidad.items():
    print(f" - {k}: {v}")

print("\nConsistencia:")
for k, v in consistencia.items():
    print(f" - {k}: {v}")

print("\nValidez:")
for k, v in validez.items():
    print(f" - {k}: {v}")

print("\nActualidad:")
for k, v in actualidad.items():
    print(f" - {k}: {v}")


Indicadores de Calidad - Tipo de Cambio SUNAT (últimos 2 meses)

Completitud:
 - Total registros: 13
 - Fechas únicas: 13
 - Nulos en campos: {'compra': 0, 'venta': 0, 'origen': 0, 'moneda': 0, 'fecha': 0}
 - Porcentaje de días sin datos: 78.69

Unicidad:
 - Duplicados por fecha: 0
 - ¿Fechas únicas = registros?: True

Consistencia:
 - Tipo de cambio < 2.0 (inválido): 0
 - Tipo de cambio > 5.0 (inválido): 0
 - Compra mayor que venta: 0

Validez:
 - Mínimo valor de venta: 3.641
 - Máximo valor de venta: 3.728
 - Mínimo valor de compra: 3.628
 - Máximo valor de compra: 3.715

Actualidad:
 - Primera fecha: 2025-01-27 00:00:00
 - Última fecha: 2025-03-28 00:00:00
 - Cantidad de días cubiertos: 61
